# Available digest modules:
cif2png_utils, tif2png_utils, montage2png_utils, tif2tensor_utils, cif2tensor_utils

In [1]:
import cif2tensor_utils

Using TensorFlow backend.


In [2]:
import keras
import numpy
import matplotlib.pyplot
import os.path
import pandas
import seaborn
import sklearn.metrics
import keras.applications
import keras.preprocessing.image
import tensorflow

# User's settings

In [3]:
directory = "/data1/Minh/IFC/DeepLearning/BFDFDAPI_digested_except113_190_207_209"

data = {
    "normal": '/data1/Minh/IFC/DeepLearning/cif/normal/',
    
  "leukemic": '/data1/Minh/IFC/DeepLearning/cif/abnormal/'
}

# Warning: Neural networks often require a combination of 1 or 3 or 4 channels.
# Users should specify ALL desired channels here. In downstream modules, users can duplicate channels if needed.
channels = [0,5,6]

image_size = 28

split = {
    "Training" : 0.8,
    "Validation" : 0.1,
    "Testing" : 0.1
}

classes = len(data.keys())

# Save 209pres data to tensor:

In [ ]:
# cif2tensor_utils.parse(directory, data, channels, image_size, split)
# class_weights = numpy.load(os.path.join(directory, "class_weights.npy"))

In [ ]:
directory = "/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test209pres"

data = {
    "normal": '/data1/Minh/IFC/DeepLearning/cif/normal/LK209pres/',
    
  "leukemic": '/data1/Minh/IFC/DeepLearning/cif/abnormal/LK209pres/',

}

# Warning: Neural networks often require a combination of 1 or 3 or 4 channels.
# Users should specify ALL desired channels here. In downstream modules, users can duplicate channels if needed.
channels = [0,5,6]

image_size = 28

split = {
    "Training" : 0,
    "Validation" : 0,
    "Testing" : 1
}

classes = len(data.keys())

# Save 190pres data to tensor:

In [ ]:
# cif2tensor_utils.parse(directory, data, channels, image_size, split)
# class_weights = numpy.load(os.path.join(directory, "class_weights.npy"))

In [ ]:
directory = "/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test190pres"

data = {
    "normal": '/data1/Minh/IFC/DeepLearning/cif/normal/LK190pres/',
    
  "leukemic": '/data1/Minh/IFC/DeepLearning/cif/abnormal/LK190pres/',

}

# Warning: Neural networks often require a combination of 1 or 3 or 4 channels.
# Users should specify ALL desired channels here. In downstream modules, users can duplicate channels if needed.
channels = [0,5,6]

image_size = 28

split = {
    "Training" : 0,
    "Validation" : 0,
    "Testing" : 1
}

classes = len(data.keys())

# Save digested data to tensor:

In [ ]:
# cif2tensor_utils.parse(directory, data, channels, image_size, split)
# class_weights = numpy.load(os.path.join(directory, "class_weights.npy"))

Adjust class weights if needed

In [ ]:
# class_weights = {0: 0.9708737192631808, 1: 0.029126280736819173}

In [ ]:
# for k in class_weights.keys():
#     class_weights[k] = 1/class_weights[k]
# print(class_weights)

In [ ]:
# Light weights
# class_weights = {0: 58.28125, 
#                  1: 2.3065652689597433, 
#                  2: 3.843739100104639, 
#                  3: 19.89130434782609, 
#                  4: 14.554455445544555, 
#                  5: 3.837047353760446, 
#                  6: 47.58928571428572}

In [4]:
# Heavy weights
class_weights = {0: 1.0300000712336965, 1: 34.333254184969725}

Visualize saved data:

In [ ]:
%matplotlib inline
import skimage.io

In [ ]:
training_x = numpy.load(os.path.join(directory, "training_x.npy"))
training_y = numpy.load(os.path.join(directory, "training_y.npy"))

In [ ]:
skimage.io.imshow(training_x[43,:,:,2], cmap = "gray")

In [ ]:
skimage.io.imshow(training_x[100,:,:,2], cmap = "gray")

# Save disgested data to PNG:
Warning: PNG files are single channel images, randomly shuffled and saved into respected "Training", "Validation", "Testing" subfolders

In [ ]:
# tif2png_utils.parse(directory, data, channels, image_size)

# tif2png_utils.split(directory, data.keys(), split, image_size)

# class_weights = tif2png_utils.class_weights("/home/minh-doan/Cell_cycle/temp_processed/Training/", data)

# Load data and labels:

In [5]:
# Use this function to normalize signal intensities across images
def min_max_norm(x, minimum=None, maximum=None):
    if minimum is None:
        minimum = x.min()
    if maximum is None:
        maximum = x.max()
    result = 100.0*( (numpy.ndarray.astype(x, numpy.float32) - minimum)/(maximum - minimum) )
    return (result, minimum, maximum)

In [6]:
training_x = numpy.load(os.path.join(directory, "training_x.npy"))

training_y = numpy.load(os.path.join(directory, "training_y.npy"))

# If using networks (VGG19) that require 3 channels RGB not single-channel grayscale, duplicate channels if needed:
# Examples:
# training_xx = numpy.concatenate((training_x,training_x,training_x), axis=3)
# training_xx = numpy.concatenate((training_x,numpy.expand_dims(training_x[:,:,:,0], axis = 3)), axis=3)
# training_xx = numpy.concatenate((training_x,training_x), axis=3)

In [7]:
# Use this function to normalize signal intensities across images
training_x, pix_min, pix_max = min_max_norm(training_x)

In [8]:
training_generator = keras.preprocessing.image.ImageDataGenerator(rotation_range = 180, horizontal_flip = True, vertical_flip = True)

training_generator = training_generator.flow(
    x = training_x, # or training_xx
    y = training_y,
    batch_size=32
)

# If using PNG from folders:

# training_generator = training_generator.flow_from_directory(
#     batch_size=32,
#     color_mode="rgb",
#     directory="/home/minh-doan/Cell_cycle/temp_processed/Training/",
#     target_size=(224, 224)
# )

In [9]:
validation_x = numpy.load(os.path.join(directory, "validation_x.npy"))

validation_y = numpy.load(os.path.join(directory, "validation_y.npy"))

# If using networks (VGG19) that needs 3 channels RGB not single-channel grayscale:
# Examples:
# validation_xx = numpy.concatenate((validation_x,validation_x,validation_x), axis=3)
# validation_xx = numpy.concatenate((validation_x,numpy.expand_dims(validation_x[:,:,:,1], axis = 3)), axis=3)
# validation_xx = numpy.concatenate((validation_x,validation_x), axis=3)

In [10]:
# Use this function to normalize signal intensities across images
validation_x, pix_min, pix_max = min_max_norm(validation_x, pix_min, pix_max)

In [11]:
validation_generator = keras.preprocessing.image.ImageDataGenerator() #rotation_range = 180, horizontal_flip = True, vertical_flip = True)

validation_generator = validation_generator.flow(
    x = validation_x, # or validation_xx
    y = validation_y,
    batch_size=32
)

# If using PNG from folders:

# validation_generator = validation_generator.flow_from_directory(
#     batch_size=32,
#     color_mode="rgb",
#     directory="/home/minh-doan/Cell_cycle/temp_processed/Validation/",
#     target_size=(224, 224)
# )

In [12]:
testing_x = numpy.load(os.path.join(directory, "testing_x.npy"))

testing_y = numpy.load(os.path.join(directory, "testing_y.npy"))

# If using networks (VGG19) that needs 3 channels RGB not single-channel grayscale:
# Examples:
# testing_xx = numpy.concatenate((testing_x,testing_x,testing_x), axis=3)
# testing_xx = numpy.concatenate((testing_x,numpy.expand_dims(testing_x[:,:,:,0], axis = 3)), axis=3)
# testing_xx = numpy.concatenate((testing_x,testing_x), axis=3)

In [13]:
# Use this function to normalize signal intensities across images
testing_x, pix_min, pix_max = min_max_norm(testing_x, pix_min, pix_max)

In [14]:
test_generator = keras.preprocessing.image.ImageDataGenerator() #rotation_range = 180, horizontal_flip = True, vertical_flip = True)

test_generator = test_generator.flow(
    x = testing_x, # or testing_xx
    y = testing_y,
    batch_size=32
)

# If using PNG from folders:

# test_generator = test_generator.flow_from_directory(
#     batch_size=1,
#     color_mode="rgb",
#     directory="/home/minh-doan/Cell_cycle/temp_processed/Testing/"
# )

# Construct convolutional neural network:

In [15]:
shape = (image_size, image_size, 3)

x = keras.layers.Input(shape)

In [16]:
options = {"activation": "relu", "kernel_size": (3, 3), "padding": "same"}

# Block 1:

y = keras.layers.Conv2D(32, **options)(x)
y = keras.layers.Conv2D(32, **options)(y)

y = keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same')(y)

# Block 2:
y = keras.layers.Conv2D(64, **options)(y)
y = keras.layers.Conv2D(64, **options)(y)

y = keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same')(y)

# Block 3:
y = keras.layers.Conv2D(128, **options)(y)
y = keras.layers.Conv2D(128, **options)(y)

y = keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same')(y)

# Block 4:
# y = keras.layers.Conv2D(256, **options)(y)
# y = keras.layers.Conv2D(256, **options)(y)

# y = keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same')(y)

# Block 5:
y = keras.layers.Flatten()(y)

intermediate_layer = keras.layers.Dense(1024, activation="relu")(y) # This intermediate_layer will be used for embeddings

y = keras.layers.Dropout(0.5)(intermediate_layer)

y = keras.layers.Dense(classes)(y)

y = keras.layers.Activation("softmax")(y)

In [17]:
model = keras.models.Model(x, y)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [19]:
loss = keras.losses.categorical_crossentropy

optimizer = keras.optimizers.Adam(0.00001)

model.compile(
    loss=loss, 
    metrics=[
        "accuracy"
    ],
    optimizer=optimizer
)

# Adapt pretrained VGG19 network

In [ ]:
# Change here the number of channesl: for VGG19 expecting (224,224,3) with 3 channels, for homemade model it's (image_size, image_size, 1)
shape = (224, 224, 3)

In [ ]:
VGG19 = keras.applications.VGG19(include_top = False, classes = classes)

In [ ]:
for layer in VGG19.layers:
    layer.trainable = False
    
x = VGG19.output
x = keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
# For classification:
# x = keras.layers.Dense(4096, activation='relu')(x)
# x = keras.layers.BatchNormalization()(x)

# For Tensorboard embedding:
intermediate_layer = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.BatchNormalization()(intermediate_layer)
x = keras.layers.Dropout(rate=0.5)(x)

predictions = keras.layers.Dense(classes, activation='softmax')(x)

# this is the model we will train
model = keras.models.Model(input=VGG19.input, output=predictions)

In [ ]:
# model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr = 0.0001)

loss = keras.losses.categorical_crossentropy

metrics = [
    keras.metrics.categorical_accuracy
]

model.compile(optimizer, loss, metrics)

# Train the network

In [20]:
# New output directory:
directory = "/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_test209_190pres_heavyweights"
if not os.path.exists(directory):
    os.makedirs(directory)

In [21]:
csv_logger = keras.callbacks.CSVLogger(os.path.join(directory, 'training.csv') )

early_stopping = keras.callbacks.EarlyStopping(patience=64)

# checkpoint
filepath = os.path.join(directory, "weights.best.hdf5")
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

In [22]:
configuration = tensorflow.ConfigProto()

configuration.gpu_options.allow_growth = True

session = tensorflow.Session(config=configuration)

keras.backend.set_session(session)

In [23]:
with tensorflow.device("/gpu:2"):
    model.fit_generator(
        callbacks=[
            #checkpoint,
            csv_logger
        ],
        epochs=15,
        class_weight = class_weights,
        generator=training_generator,
        max_q_size=256,
        steps_per_epoch=2000,
        validation_data=validation_generator,
        validation_steps=2000
    )

Epoch 1/15
2000/2000 [==============================] - 35s - loss: 1.3976 - acc: 0.5288 - val_loss: 0.6885 - val_acc: 0.7830

Evaluate testing set

In [24]:
model.evaluate_generator(
    generator=test_generator, 
    steps=256
)

[0.9279368898132816, 0.630859375]

In [25]:
testing_x = numpy.load("/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test209pres/testing_x.npy")

testing_y = numpy.load("/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test209pres/testing_y.npy")

# If using networks (VGG19) that needs 3 channels RGB not single-channel grayscale:
# Examples:
# testing_xx = numpy.concatenate((testing_x,testing_x,testing_x), axis=3)
# testing_xx = numpy.concatenate((testing_x,numpy.expand_dims(testing_x[:,:,:,0], axis = 3)), axis=3)
# testing_xx = numpy.concatenate((testing_x,testing_x), axis=3)

In [26]:
# Use this function to normalize signal intensities across images
testing_x, pix_min, pix_max = min_max_norm(testing_x, pix_min, pix_max)

In [27]:
test_generator = keras.preprocessing.image.ImageDataGenerator() #rotation_range = 180, horizontal_flip = True, vertical_flip = True)

test_generator = test_generator.flow(
    x = testing_x, # or testing_xx
    y = testing_y,
    batch_size=32
)

# If using PNG from folders:

# test_generator = test_generator.flow_from_directory(
#     batch_size=1,
#     color_mode="rgb",
#     directory="/home/minh-doan/Cell_cycle/temp_processed/Testing/"
# )

In [28]:
model.evaluate_generator(
    generator=test_generator, 
    steps=256
)

[0.68030340862384808, 0.59225790694106673]

In [29]:
testing_x = numpy.load("/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test190pres/testing_x.npy")

testing_y = numpy.load("/home/minh-doan/Leukemia_DeepLearning/BFDFDAPI_digested_test190pres/testing_y.npy")

# If using networks (VGG19) that needs 3 channels RGB not single-channel grayscale:
# Examples:
# testing_xx = numpy.concatenate((testing_x,testing_x,testing_x), axis=3)
# testing_xx = numpy.concatenate((testing_x,numpy.expand_dims(testing_x[:,:,:,0], axis = 3)), axis=3)
# testing_xx = numpy.concatenate((testing_x,testing_x), axis=3)

In [30]:
# Use this function to normalize signal intensities across images
testing_x, pix_min, pix_max = min_max_norm(testing_x, pix_min, pix_max)

In [31]:
test_generator = keras.preprocessing.image.ImageDataGenerator() #rotation_range = 180, horizontal_flip = True, vertical_flip = True)

test_generator = test_generator.flow(
    x = testing_x, # or testing_xx
    y = testing_y,
    batch_size=32
)

# If using PNG from folders:

# test_generator = test_generator.flow_from_directory(
#     batch_size=1,
#     color_mode="rgb",
#     directory="/home/minh-doan/Cell_cycle/temp_processed/Testing/"
# )

In [32]:
model.evaluate_generator(
    generator=test_generator, 
    steps=256
)

[1.1610597355756909, 0.2862548828125]

# Generate Graph for visualization in Tensorboard

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./', histogram_freq=0, write_graph=True, write_images=True)

options = {
    "batch_size": 1,
    "class_weight": class_weights,
    "callbacks": [
        tbCallBack
    ],
    "epochs": 1,
    "verbose": 0,
    "x": testing_x,
    "y": testing_y    
}

%time model.fit(**options)

# Generate embeddings for visualization in Tensorboard

Converting one-hot labels into class names

In [ ]:
class_names = sorted(list(data.keys()))

def save_metadata(file):
    with open(file, 'w') as f:
        for i in range(testing_y.shape[0]):
            for j in range(len(class_names)):
                if numpy.nonzero(testing_y[::1])[1:][0][i] == j:
                    c = class_names[j]
                    f.write('{}\n'.format(c))
            
save_metadata('./metadata.tsv')

Converting one-hot labels into serial numbers [0,1,2,3...]

In [ ]:
# def save_metadata(file):
#     with open(file, 'w') as f:
#         for i in range(testing_y.shape[0]):
#             c = numpy.nonzero(testing_y[::1])[1:][0][i]
#             f.write('{}\n'.format(c))
            
# save_metadata('./metadata.tsv')

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector

intermediate_values = intermediate_layer.eval(feed_dict = {x : testing_x}, session = keras.backend.get_session())
print(intermediate_values.shape)

In [ ]:
embedding_var = tensorflow.Variable(intermediate_values)

# file names
ROOT_DIR = './' # Warning! If specified as absolute path, the resulted files will be BOUND to that folder path
OUTPUT_MODEL_FILE_NAME = os.path.join(ROOT_DIR,'tf.ckpt')

# keras.backend.get_session()
embedSess = tensorflow.Session()

# save variable in session
embedSess.run(embedding_var.initializer)
# embedSess.run(tensorflow.global_variables_initializer())

# save session (only used variable) to file
saver = tensorflow.train.Saver([embedding_var])
saver.save(embedSess, OUTPUT_MODEL_FILE_NAME)

summary_writer = tensorflow.summary.FileWriter(ROOT_DIR)

config = tensorflow.contrib.tensorboard.plugins.projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.metadata_path = 'metadata.tsv' # this metadata_path need to be modified later. See note.
tensorflow.contrib.tensorboard.plugins.projector.visualize_embeddings(summary_writer, config)

# Note:
Have a careful check in this output : "projector_config.pbtxt"
"/path/to/logdir/metadata.tsv" has to be specified, CANNOT be relative path "./metadata.tsv", nor "~/metadata.tsv"

Type command in terminal: tensorboard --logdir="/path/to/logdir"

Next, open web-browser, connect to http://localhost:6006.

# Categorical accuracy and loss

In [ ]:
metrics = pandas.read_csv(os.path.join(directory, 'training.csv') )

In [ ]:
print(metrics)

In [ ]:
%matplotlib inline

In [ ]:
matplotlib.pyplot.plot(metrics["acc"])
matplotlib.pyplot.plot(metrics["val_acc"])

In [ ]:
matplotlib.pyplot.plot(metrics["loss"])
matplotlib.pyplot.plot(metrics["val_loss"])

# Confusion matrix

In [ ]:
predicted = model.predict(
    batch_size=50,
    x=testing_x
#     x=testing_xx # for 3-channel RGB
)

predicted = numpy.argmax(predicted, -1)

In [ ]:
expected = numpy.argmax(testing_y[:, :], -1)

In [ ]:
%matplotlib inline

In [ ]:
confusion = sklearn.metrics.confusion_matrix(expected, predicted)

confusion = pandas.DataFrame(confusion)

matplotlib.pyplot.figure(figsize=(12, 8))

seaborn.heatmap(confusion, annot=True)

In [ ]:
predicted = model.predict(
    batch_size=50,
    x=testing_x
#     x=testing_xx # for 3-channel RGB
)

predicted = numpy.argmax(predicted, -1)

In [ ]:
expected = numpy.argmax(testing_y[:, :], -1)

In [ ]:
confusion = sklearn.metrics.confusion_matrix(expected, predicted)

confusion = pandas.DataFrame(confusion)

matplotlib.pyplot.figure(figsize=(12, 8))

seaborn.heatmap(confusion, annot=True)

In [33]:
session.close()